In [1]:
!git clone https://github.com/namwasinyourheart/french-to-english-translation.git

Cloning into 'french-to-english-translation'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 57 (delta 3), reused 13 (delta 2), pack-reused 40
Unpacking objects: 100% (57/57), 465.43 MiB | 12.13 MiB/s, done.
Updating files: 100% (36/36), done.


In [2]:
!tree french-to-english-translation

french-to-english-translation
|-- data
|   |-- _about.txt
|   |-- corpus_src.txt
|   |-- corpus_trg.txt
|   |-- eng_list_18662_cleaned.txt
|   |-- fra.txt
|   |-- nmt_src_vocab.model
|   |-- nmt_src_vocab.vocab
|   |-- nmt_trg_vocab.model
|   |-- nmt_trg_vocab.vocab
|   |-- ratings_test.txt
|   |-- ratings_test_t5.json
|   |-- ratings_train.txt
|   |-- ratings_train_t5.json
|   |-- vi_en_681.csv
|   `-- zip
|       `-- fra-eng.zip
|-- notebooks
|   |-- A1_TF2_NMT_T5_fr_en_Keras_Tokenizer\ (1).py
|   |-- A1_TF2_NMT_T5_fr_en_Keras_Tokenizer.ipynb
|   |-- A1_TF2_NMT_T5_fr_en_Keras_Tokenizer.py
|   |-- A6_Torch_NMT_T5_fr_en_Keras_Tokenizer.py
|   |-- A8_Torch_NMT_T5_fr_en_Sentencepiece.py
|   |-- Pytorch-A1_TF2_NMT_T5_fr_en_Keras_Tokenizer.ipynb
|   |-- finetune-T5-fre-en.ipynb
|   `-- transformer
`-- save
    `-- checkpoints
        |-- checkpoint
        |-- ckpt-10.data-00000-of-00001
        |-- ckpt-10.index
        |-- ckpt-6.data-00000-of-00001
        |-- ckpt-6.index
        |-- c

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, T5Tokenizer, T5ForConditionalGeneration, MT5Tokenizer, MT5ForConditionalGeneration
import time
import random
import numpy as np

In [4]:
import pandas as pd
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

In [5]:
!pip install rouge
from rouge import Rouge

In [6]:
!pip install sacrebleu
import sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 5.1 MB/s eta 0:00:00


In [7]:
def set_seed(
    seed: int = 42, deterministic: bool = True, benchmark: bool = False
) -> None:

    random.seed(seed)
    np.random.seed(seed)
    torch.random.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

        torch.cuda.deterministic = deterministic
        torch.cuda.benchmark = benchmark

In [8]:
SEED = 20240404
set_seed(SEED)

In [9]:
data_df = pd.read_csv('/kaggle/working/french-to-english-translation/data/vi_en_681.csv')
data_df.rename(columns={'VI': 'SRC', 'EN': 'TRG'}, inplace=True)
data_df

,SRC,TRG
0,Chính trong một trận động đất lớn đã phá hủy T...,it was in the s that a big earthquake destroye...
1,Bạn có thể đưa cho tôi tờ báo trên bàn không?,could you hand me the newspaper on the table ?
2,Tôi phải làm tốt nhất căn phòng nhỏ đó.,i have to make the best of that small room .
3,Một người chú là anh trai của một trong những ...,an uncle is the brother of one of your parents .
4,Anh ta mất gần như tất cả những con tem anh ta...,he lost almost all the stamps he had collected .
...,...,...
676,Tôi giữ một nguồn cung cấp tốt để lưu các chuy...,i keep a good supply of stamps to save trips t...
677,Mọi người làm việc cho chúng tôi kiếm được nhi...,everyone working for us earns more than the mi...
678,Anh ấy đã làm hết sức mình và vẫn có điểm kém.,he did his best and still had poor marks .
679,Tom nghĩ rằng anh ta có thể thành công nơi nhữ...,tom thinks he can succeed where others have fa...


In [10]:
set_seed(SEED)
train_df, test_df = train_test_split(data_df, test_size=0.1,random_state=SEED)
train_df, val_df = train_test_split(train_df, test_size=1/9, random_state=SEED) 

In [17]:
# '''
# D9. Add <SOS>, <EOS> for source and target
# '''

# train_df['SRC'].apply(lambda x: "<SOS> " + str(x) + " <EOS>")
# train_df['TRG'].apply(lambda x: "<SOS> " + str(x) + " <EOS>")


In [11]:
len(train_df), len(val_df), len(test_df)

(544, 68, 69)

# Extend vocab

In [12]:
from collections import Counter

# Tokenize the Vietnamese sentences to words
all_words = Counter(" ".join(data_df["SRC"].tolist()).split())

# Display the size of the vocabulary and some of the most common words
vocabulary_size = len(all_words)
most_common_words = all_words.most_common(10)

vocabulary_size, most_common_words


(1699,
 [('tôi', 277),
  ('một', 200),
  ('có', 176),
  ('Tôi', 173),
  ('không', 173),
  ('bạn', 155),
  ('đã', 136),
  ('của', 124),
  ('thể', 107),
  ('là', 103)])

In [13]:
all_words

Counter({'tôi': 277,
         'một': 200,
         'có': 176,
         'Tôi': 173,
         'không': 173,
         'bạn': 155,
         'đã': 136,
         'của': 124,
         'thể': 107,
         'là': 103,
         'cho': 93,
         'với': 93,
         'sẽ': 91,
         'anh': 82,
         'trong': 78,
         'để': 75,
         'khi': 72,
         'làm': 70,
         'nói': 69,
         'ấy': 68,
         'Tom': 68,
         'chúng': 68,
         'rằng': 66,
         'ta': 63,
         'và': 59,
         'đến': 58,
         'Bạn': 54,
         'phải': 54,
         'người': 53,
         'ở': 52,
         'đi': 52,
         'muốn': 48,
         'những': 47,
         'gì': 47,
         'Anh': 43,
         'điều': 42,
         'như': 40,
         'được': 40,
         'cô': 38,
         'đó': 38,
         'tôi.': 37,
         'thấy': 36,
         'vào': 35,
         'hơn': 34,
         'nghĩ': 33,
         'đang': 31,
         'nhưng': 31,
         'về': 30,
         'Chúng': 30,
  

In [44]:
sorted_words = all_words.most_common()

sorted_words_list = [word for word, count in sorted_words]
new_tokens_1 = sorted_words_list
len(new_tokens_1)

1699

In [39]:
tokenizer = T5Tokenizer.from_pretrained('google-t5/t5-small')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [40]:
# t5_vocab = set(tokenizer.get_vocab())
print("Vocab size before adding new tokens:", len(tokenizer))

Vocab size before adding new tokens: 32100


In [ ]:
new_tokens = set(new_vietnamese_words) - set(tokenizer.get_vocab())
len(new_tokens), new_tokens

In [42]:
tokenizer.add_tokens(list(new_tokens))
print("Vocab size before adding new tokens:", len(tokenizer))

Vocab size before adding new tokens: 33759


In [66]:
# print(tokenizer.convert_tokens_to_ids('jean'))
# # print(tokenizer.convert_ids_to_tokens(1313))

# print('y' in new_vietnamese_words)

In [21]:
t5_small = T5ForConditionalGeneration.from_pretrained('google-t5/t5-small')

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [22]:
model = t5_small
model.shared.num_embeddings

32128

In [23]:
# Resize the embeddings layer to match the tokenizer's vocabulary size
model.resize_token_embeddings(len(tokenizer))

# Get the size of the embeddings layer after resizing
model.shared.num_embeddings

33759

In [24]:
model.state_dict()['shared.weight'].shape

torch.Size([33759, 512])

In [43]:
def extend_tkn_vocab(tokenizer, new_tokens):
    print("Vocab size before adding new tokens:", len(tokenizer))
    
    new_tokens = set(new_tokens) - set(tokenizer.get_vocab())
    
    tokenizer.add_tokens(list(new_tokens))
    print("Vocab size before adding new tokens:", len(tokenizer))
    
    return tokenizer
    

# Datasets

In [33]:
class Vi2EnDataset(Dataset):
    def __init__(self, dataframe, tokenizer, source_len, target_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.target_len = target_len
        
        self.src_texts = dataframe['SRC'].tolist()
        self.trg_texts = dataframe['TRG'].tolist()
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, idx):
        
        src_text = self.src_texts[idx]
        trg_text = self.trg_texts[idx]
        
        source = self.tokenizer.batch_encode_plus(
            [src_text],
            max_length=self.source_len,
            pad_to_max_length=True,
            truncation=True,
            return_tensors="pt",
        )
        
        target = self.tokenizer.batch_encode_plus(
            [trg_text],
            max_length=self.target_len,
            pad_to_max_length=True,
            truncation=True,
            return_tensors="pt",
        )
        
        source_ids = source['input_ids'].squeeze()
        source_mask = source["attention_mask"].squeeze()
        
        target_ids = target['input_ids'].squeeze()
        target_mask = target["attention_mask"].squeeze()
        
        
        return {
            "source_ids": source_ids,
            "source_mask": source_mask,
            "target_ids": target_ids,
            "target_ids_y": target_ids,
        }
        

In [78]:
# Create Data Loaders
BATCH_SIZE = 4
max_len = 41

# tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
# model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

# train_dataset = Vi2EnDataset(train_df, tokenizer, max_len, max_len)
# train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# val_dataset = Vi2EnDataset(val_df, tokenizer, max_len, max_len)
# val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,shuffle=False)

# test_dataset = Vi2EnDataset(test_df, tokenizer, max_len, max_len)
# test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,shuffle=False)

# Function to train

In [74]:
def train(epoch, model, tokenizer, loader, optimizer, device):

    """
    Function to be called for training with the parameters passed from main function

    """
    total_loss = 0
    model = model.to(device)       
    
    model.train()
                     
    for batch_idx, batch in enumerate(loader, 1):
        y = batch["target_ids"].to(device)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        
        source_ids = batch["source_ids"].to(device)
        source_mask = batch["source_mask"].to(device)

        outputs = model(
            input_ids=source_ids,
            attention_mask=source_mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
        )
        loss = outputs[0]
        total_loss += loss.item()
                     
        wandb.log({
#             "epoch": epoch,
#             "step": batch_idx,
            "train_loss": loss.item()
        })
        
        if batch_idx % 20 == 0:
            print(f"Epoch: {epoch + 1}, Batch: {batch_idx}, Loss: {loss.item()}")

#         if idx % 10 == 0:
#             training_logger.add_row(str(epoch), str(_), str(loss))
#             console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch: {epoch + 1}, Average Loss: {avg_train_loss:.3f}")
    
    return avg_train_loss


In [84]:
def validate(epoch, model, tokenizer, loader, results_file, device):
    model.eval()
    total_loss = 0
    predictions = []
    references = []
    
    model = model.to(device)

    with torch.no_grad():
        for batch in tqdm(loader):
            y = batch["target_ids"].to(device)
            y_ids = y[:, :-1].contiguous()
            lm_labels = y[:, 1:].clone().detach()
            lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
            
            source_ids = batch["source_ids"].to(device)
            source_mask = batch["source_mask"].to(device)

            outputs = model(
                input_ids=source_ids,
                attention_mask=source_mask,
                decoder_input_ids=y_ids,
                labels=lm_labels,
            )
            loss = outputs[0]
            total_loss += loss.item()

            # For BLEU and ROUGE score calculations
            preds = torch.argmax(outputs.logits, dim=-1)
            
            predictions.extend([tokenizer.decode(pred, skip_special_tokens=True) for pred in preds])
            references.extend([tokenizer.decode(ref, skip_special_tokens=True) for ref in batch["target_ids"]])
            


    avg_loss = total_loss / len(loader)
    print(f"Validation Loss Epoch: {epoch + 1}: {avg_loss:.2f}")
    
    for i in range(10):
        print('------'*4)
        print(predictions[i])
        print(references[i])
        print('------'*4)
        print('\n')

    # Calculate BLEU score
    bleu_score = sacrebleu.corpus_bleu(predictions, references)
    print(f"Validation BLEU Score Epoch: {epoch + 1}: {bleu_score.score:.2f}")
    
    try: 
        rouge = Rouge()
        rouge_scores = rouge.get_scores(predictions, references, avg=True)
        rouge_1_score = round(rouge_scores['rouge-1']['f'],2)
        rouge_2_score = round(rouge_scores['rouge-2']['f'],2)
        rouge_l_score = round(rouge_scores['rouge-l']['f'],2)
    except: 
        rouge_scores = rouge_1_score = rouge_2_score = rouge_l_score = "ERROR"
    print(f"Validation ROUGE Scores Epoch: {epoch + 1}: ROUGE-1={rouge_1_score}, ROUGE-2={rouge_2_score}, ROUGE-L={rouge_l_score}")
    
#     results_file.write(f"{bleu_score.score}\t{rouge_1_score}\t{rouge_2_score}\t{rouge_l_score}\n")

    
    return avg_loss, bleu_score.score, rouge_scores

   


In [65]:
train_flags = [True]

In [47]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [27]:
import wandb

In [30]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [53]:
wandb.init(project="vi-en-translation",  
           name="exp2_extend-tkn-vocab")

wandb: Currently logged in as: namfam (paracetamol). Use `wandb login --relogin` to force relogin


In [31]:
# %env CUDA_LAUNCH_BLOCKING=1

In [38]:
# for batch in train_loader:
#     print(batch['source_ids'])

In [ ]:
# for epoch in range(1000):
#     train(epoch, model, tokenizer, train_loader, optimizer, device)
#     validate(epoch, model, tokenizer, val_loader, results_file, device)

In [76]:
models_to_try = {
#     "google-t5/t5-small": (T5ForConditionalGeneration, T5Tokenizer),
#     "google-t5/t5-base": (T5ForConditionalGeneration, T5Tokenizer),
#     'google/mt5-small': (MT5ForConditionalGeneration, MT5Tokenizer),
    "google-t5/t5-base": (T5ForConditionalGeneration, T5Tokenizer),
}



In [ ]:
SEED = 20240404
set_seed(SEED)

num_epochs = 200
# results_file_name = f"experiment_results_{timestamp}.txt"
# results_file_name

with open(results_file_name, "w") as results_file:
    results_file.write("Model\tTokenizer\tTrain?\tBLEU Score\tROUGE-1\tROUGE-2\tROUGE-L\n")
    

    
    for model_name, (model_class, tokenizer_class) in models_to_try.items():
        
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        wandb.init(project="vi-en-translation",  
                    name=f"exp2_{model_name}_extend-tkn-vocab_{timestamp}")

        for train_flag in train_flags:  # Train and Not Train
            
            
            print(f"Experimenting with {model_name}, Train={train_flag}")
            results_file.write(f"{model_name}\t{model_name}\t{train_flag}\t")

            tokenizer = tokenizer_class.from_pretrained(model_name)
            tokenizer = extend_tkn_vocab(tokenizer, new_tokens_1)
            
            model = model_class.from_pretrained(model_name)
            model.resize_token_embeddings(len(tokenizer))
            

            train_dataset = Vi2EnDataset(train_df, tokenizer, max_len, max_len)
            train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

            val_dataset = Vi2EnDataset(val_df, tokenizer, max_len, max_len)
            val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,shuffle=False)

            test_dataset = Vi2EnDataset(test_df, tokenizer, max_len, max_len)
            test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE,shuffle=False)



            if train_flag:
                optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) 
                # Train the model
                for epoch in range(num_epochs):
                    epoch += 1
                    avg_train_loss = train(epoch, model, tokenizer, train_loader, optimizer, device)
    
                    val_loss, bleu_score, rouge_score = validate(epoch, model, tokenizer, val_loader, results_file, device)
                    wandb.log({
                        "val_loss": val_loss,
                        "bleu_score": bleu_score,
#                         "rouge_score": rouge_score
                    })
            
                wandb.save(f"{model_name.replace('/', '-')}_epoch_{num_epochs}.pt")
                wandb.finish()
            else:
                epoch = -1
                avg_val_loss, bleu_score, rouge_score = validate(epoch, model, tokenizer, val_loader, results_file, device)
                wandb.log({"val_loss": avg_val_loss})
                
            

            
#             print(f"BLEU Score: {bleu_score}")
#             print(f"ROUGE Scores: {rouge_score}")

train_loss,█▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▁▁▂▁▁▁▂▁▁▁▁▁
train_loss,4.39762


Experimenting with google-t5/t5-base, Train=True


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Vocab size before adding new tokens: 32100
Vocab size before adding new tokens: 33759
Epoch: 2, Batch: 20, Loss: 8.195303916931152
Epoch: 2, Batch: 40, Loss: 6.192114353179932
Epoch: 2, Batch: 60, Loss: 5.554837703704834
Epoch: 2, Batch: 80, Loss: 4.933018207550049
Epoch: 2, Batch: 100, Loss: 4.802083969116211
Epoch: 2, Batch: 120, Loss: 5.167352199554443
Epoch: 2, Average Loss: 6.276


  0%|          | 0/17 [00:00<?, ?it/s]

...

  0%|          | 0/17 [00:00<?, ?it/s]

Validation Loss Epoch: 24: 4.27
------------------------
wave beena new   very from thes ho ho ol for. so so sofa. sofa.
she has a cold and is absent from sc ho ol.
------------------------


------------------------
has trying to get  to ck for anw ryution. sofa. sofa. sofa. sofa. sofa. sofa. sofa. sofa.
the government is trying to bring things ba ck to nor ma l.
------------------------


------------------------
is còn s to t he is in he is d yes about.
it se em s tha t he believes what he sai d is right.
------------------------


------------------------
can have to quite me whatw ho were to to go tochrist you
you still have not told me w hy you decided not to go.
------------------------


------------------------
i camesa d the area use of  old on the the. car . the the the so so so so so so so so so inch. inch. inch. inch.
he e xa mined the ho use with an eye to opening a store there.
------------------------


------------------------
wavees all the channels and see if  car is

  0%|          | 0/17 [00:00<?, ?it/s]

Validation Loss Epoch: 25: 4.33
------------------------
asked beena new    from thes ho ho ol for. so so
she has a cold and is absent from sc ho ol.
------------------------


------------------------
has trying to get  to n to an  ryution.
the government is trying to bring things ba ck to nor ma l.
------------------------


------------------------
is còn d to t toi is in he is d yes about. what
it se em s tha t he believes what he sai d is right.
------------------------


------------------------
can have to quite me whatw ho were to to go tochrist you
you still have not told me w hy you decided not to go.
------------------------


------------------------
i camesa d the area use   old on the the. car . the the the so so so so so so so so so so so
he e xa mined the ho use with an eye to opening a store there.
------------------------


------------------------
wavees all the  and see if  car is on it. she so so so so so so
she skimmed through the register to see if her name was i

  0%|          | 0/17 [00:00<?, ?it/s]

In [69]:
def translate_texts(model, 
                    tokenizer, 
                    texts, 
                    device, 
                    source_len = 41, 
                    target_len = 41):
    model.eval()
    translations = []

    with torch.no_grad():
        for text in texts:
            inputs = tokenizer.encode_plus(text, 
                                           return_tensors='pt', 
                                           max_length=source_len, 
                                           truncation=True, 
                                           padding="max_length").to(device)
            
            output_sequences = model.generate(input_ids=inputs['input_ids'], 
                                              attention_mask=inputs['attention_mask'], 
                                              max_length=target_len)
            translation = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
            translations.append(translation)
    
    return translations

In [ ]:
texts = ["bạn và tôi", "xin chào, "đã đến lúc để tôi huỷ kế hoạch của bạn"]
translated_texts = translate_texts(model, tokenizer, texts, device)
for src, trg in zip(texts, translated_texts):
    print(f"Source: {src}\nTranslation: {trg}\n")